# Restaurant Recommendations

In this notebook we generate meal recommendations for a Zess App registered user who has a history of previous restaurant or meal selections (likes & dislikes). Restaurant recommendations for the recommended meals are given. If the user has a history of allergen(s), recommendations are given that filter out meals containing said allergen(s).

*** 

Therefore, 2 use-case scenarios are given for the recommendations:

* The user **doesn't** have any **dietary needs/allergies/intolerances**. Therefore, recommendations are generated based upon user preference & taste alone.

* The user **does** have **dietary needs/allergies/intolerances**. Recommendations are thus generated based upon a mix of user preference, taste, and dietary needs.


# 1. User preference and taste based restaurant recommendations

### Step 1
*We will first import the required Python modules*

In [1]:
import pickle
import json
import sys
sys.path.append("../")
import numpy as np
import torch
from train import evaluation
import postprocess
from typedb.client import *
import typedb as td
import pprint

def numbered_print(list_item):
    for i, a in enumerate(list(set(list_item))):
        print(f"\033[1m{i+1}.)\033[0m {a}")
    print("\n")

def replace_last(string, delimiter, replacement):
    start, _, end = string.rpartition(delimiter)
    return start + replacement + end

ratings_filepath = "../../data/restaurant/raw_ratings.txt"
model_filepath = "../../data/py_model.pkl"
client_address = '127.0.0.1:1729'

### Step 2
*Now we load the trained deep learning model. This includes the user (test data) data that we wish to generate recommendations for.*

In [2]:
model = torch.load(model_filepath)

In [3]:
test_data, model_parameters = pickle.load(open("../save.pkl", "rb"))

### Step 3
*Here we use the trained model to generate used based recommendation predictions*

In [4]:
_, _, test_scores = evaluation(
            model, test_data, model_parameters
        )

### Step 4
*Now we load the menu-items (i.e. a dish) and their respective encoded values mapping data file.*

In [5]:
with open('../../data/restaurant/encoded_mapping.txt') as f:
    mapping = json.load(f)

### Step 5
*Now we need to select the ID of the user to fetch user data and generate recommendations for said user. That includes the appropriate meals/dishes for the user. This also includes meal to restaurant mapping data. We will then postprocess the model predictions.*

In [6]:
"""
Select user with id: 100
"""

user_id = 1000

results_dict = postprocess.postprocess_predictions(test_data, test_scores)
recommendations_dict = postprocess.generate_recommendations_dict(results_dict)
test_menu_items = postprocess.modify_recommendations_dict(recommendations_dict, mapping, user_id)
recommendations = postprocess.out(test_menu_items)

### User based recommendations
<div class="alert alert-block alert-success">
<b>Success:</b> Now we can see the users recommended meals!
</div>

In [7]:
print(f"\n\033[1mRecommended Menu Items for user with user id {user_id}\033[0m:\n ")
numbered_print(recommendations)


Recommended Menu Items for user with user id 1000:
 
1.) Red Wine Spaghetti with Meatballs
2.) Ice Cream Eggnog
3.) Mexican Rhubarb Chocolate Chunk Brownies
4.) Portobello Burger
5.) Duck Confit
6.) Grilled Tofu and Chicken Pad Thai




***
Let's take a look at what they have previously liked and disliked

In [8]:
user_old_ratings = postprocess.read_ratings_json(ratings_filepath)
previous_liked_items, previous_disliked_items = postprocess.user_past_ratings(user_old_ratings, recommendations, user_id)
print("\033[1mMenu Items the user has previously liked:\033[0m \n")
numbered_print(previous_liked_items)
print("\033[1mMenu Items the user has previously disliked:\033[0m \n")
numbered_print(previous_disliked_items)

Menu Items the user has previously liked: 

1.) Bacon Cheeseburgers with Spicy Mayonnaise
2.) Rib-Eye Steak With Herb Butter and Charred Peppers
3.) Pasta Primavera
4.) Spaghetti with a Twist
5.) Profiteroles
6.) Linguine with Sun-Dried Tomatoes
7.) Handcrafted Mushroom Lasagna
8.) California Caponata
9.) Grilled Zucchini and Herb Pizza
10.) Italian Marinated Sirloin Steak
11.) Spinach and Mushroom Lasagna


Menu Items the user has previously disliked: 

1.) Salmon Florentine
2.) Green Shrimp Lo Mein
3.) Thai Fish Curry
4.) B.B.Q. Garlic Crab
5.) Grilled Peaches with Prosciutto and Balsamic
6.) Upside-Down Apple Skillet Pie
7.) Tuna Tartare
8.) Tofu and Peanut Stir-Fry
9.) Ahi Tuna with Napa Cabbage Salad
10.) Dark Chocolate-Coconut Fondue
11.) Vegetable Tart
12.) Buffalo Patatas Bravas
13.) Mussels and Clams with Spicy Tomato Broth




In [9]:
restaurant_dict = {}
for menu_item in recommendations:
    query = 'match $mi isa menu-item, has name $min, has restaurant $mir; { $min contains "' + menu_item + '";}; '
    query += f'get $mir; offset 0; limit 1;'
    with td.client.TypeDB.core_client(client_address) as client:
        with client.session('Q3', SessionType.DATA) as session:
            with session.transaction(TransactionType.READ) as read_transaction:
                answer_iterator = read_transaction.query().match(query)
                for i, answer in enumerate(answer_iterator):
                    restaurant = answer.get('mir')
                    restaurant_dict[menu_item] = restaurant.get_value()

### Outcome
The user can therefore go to the following restaurants for their recommended meals:

In [10]:
print('\n\n')
d = {n:[k for k in restaurant_dict.keys() if restaurant_dict[k] == n] for n in set(restaurant_dict.values())}
                    
for i, (k, v) in enumerate(d.items()):
    if len(v) > 1:
        items = ", ".join(v)
        items = replace_last(items, ',', ', &')
        print(f"\033[1m{i+1}.)\033[0m \033[94m{items}\033[0m are served at \033[1m{k}\033[0m\n")
    else:
        print(f"\033[1m{i+1}.)\033[0m \033[94m{v[0]}\033[0m is served at \033[1m{k}\033[0m\n")
print('\n\n')




1.) Red Wine Spaghetti with Meatballs is served at Bubba Gump Shrimp Company

2.) Portobello Burger is served at Chuck-A-Rama

3.) Grilled Tofu and Chicken Pad Thai is served at Cabalen

4.) Duck Confit is served at Cactus Club Cafe

5.) Mexican Rhubarb Chocolate Chunk Brownies is served at The Counter

6.) Ice Cream Eggnog is served at HopCat






# 2. User preference, taste, and allergen friendly based restaurant recommendations
***

Here we will include allergen preferences for the user and query the knowledge-graph again to determine what restaurants they can go to for their recommended meals.

<div class="alert alert-block alert-info">
<b>Defining the allergens:</b> Here we define the allergens in a list from the statutory allergen requirements (14 allergens), including eggs, fish, lupin, milk, molluscs, Cereals containing gluten, mustard, treenuts, peanuts, seasame, soya, sulphur, celery (allergen), and crustaceans.

In this example we are searching for eggs & fish against this users recommended preferred meals to see what they can safely eat.
</div>

##### Allergens selected:
1. Eggs
2. Fish

### Step 1
*Select the allergens and query the knowledge-graph to return ingredients and their associated allergens*

In [11]:
user_allergens = ['eggs', 'fish'] # We are defining the users allergens here

In [12]:
answer_dict = {}
for menu_item in recommendations:
    query = 'match $mi isa menu-item, has name $min; { $min contains "' + menu_item + '";}; '
    query += f'$i isa ingredient, has name $in, has allergen $ia; '
    query += f'$con (ingredient-belongs-to-menu-item: $mi, menu-item-has-ingredient: $i) isa contains; '
    query += f'get $in, $ia; offset 0; limit 30;'
    with td.client.TypeDB.core_client(client_address) as client:
        with client.session('Q3', SessionType.DATA) as session:
            with session.transaction(TransactionType.READ) as read_transaction:
                
                answer_iterator = read_transaction.query().match(query)
                temp_dict = {}
                for i, answer in enumerate(answer_iterator):
                    ingredient = answer.get('in')
                    allergen = answer.get('ia')
                    temp_dict[ingredient.get_value()] = allergen.get_value()
                answer_dict[menu_item] = temp_dict

In [13]:
contains_allergens = []
for k, v in answer_dict.items():
    for allergen in user_allergens:
        if allergen in v.values():
            contains_allergens.append(k)
# pprint.pprint(answer_dict)

<div class="alert alert-block alert-success">
<b>Success!</b> We now have a list of meals containing allergens that the user is allergic to. So, we can remove them and give back a clean filtered recommendation. 
</div>

In [14]:
print(f'The following allergen containing meals have been identified from the users preferred meals:\n\n')
numbered_print(contains_allergens)
allergen_free_list = [x for x in list(answer_dict.keys()) if x not in contains_allergens]
allergen_free_list = list(set(allergen_free_list))

The following allergen containing meals have been identified from the users preferred meals:


1.) Red Wine Spaghetti with Meatballs
2.) Mexican Rhubarb Chocolate Chunk Brownies




In [15]:
print('The following preferred meals are therefore allergen free and the user may consume them:\n\n')
numbered_print(allergen_free_list)

The following preferred meals are therefore allergen free and the user may consume them:


1.) Portobello Burger
2.) Duck Confit
3.) Ice Cream Eggnog
4.) Grilled Tofu and Chicken Pad Thai




### Step 2
*Now we need to find what restaurant(s) the user can go to for their recommended meals*

<div class="alert alert-block alert-info">
<b>Eating out:</b> Now that we know what the user can eat from their predicted preferred meals, we can show them what restaurants they can go to by again querying the Knowledge Graph.
</div>

In [16]:
restaurant_allergen_dict = {}
for menu_item in allergen_free_list:
    query = 'match $mi isa menu-item, has name $min, has restaurant $mir; { $min contains "' + menu_item + '";}; '
    query += f'get $mir; offset 0; limit 1;'
    with td.client.TypeDB.core_client(client_address) as client:
        with client.session('Q3', SessionType.DATA) as session:
            with session.transaction(TransactionType.READ) as read_transaction:
                answer_iterator = read_transaction.query().match(query)
                for i, answer in enumerate(answer_iterator):
                    restaurant = answer.get('mir')
                    restaurant_allergen_dict[menu_item] = restaurant.get_value()

### Outcome
*The user can go to the restaurants shown below:*

In [17]:
print('\n\n')
d = {n:[k for k in restaurant_allergen_dict.keys() if restaurant_allergen_dict[k] == n] for n in set(restaurant_allergen_dict.values())}

for i, (k, v) in enumerate(d.items()):
    if len(v) > 1:
        items = ", ".join(v)
        items = replace_last(items, ',', ', &')
        print(f"\033[1m{i+1}.)\033[0m \033[94m{items}\033[0m are served at \033[1m{k}\033[0m\n")
    else:
        print(f"\033[1m{i+1}.)\033[0m \033[94m{v[0]}\033[0m is served at \033[1m{k}\033[0m\n")
print('\n\n')




1.) Ice Cream Eggnog is served at HopCat

2.) Grilled Tofu and Chicken Pad Thai is served at Cabalen

3.) Duck Confit is served at Cactus Club Cafe

4.) Portobello Burger is served at Chuck-A-Rama






# 3. Conclusion
<div class="alert alert-block alert-success">
As demonstrated, we can generate meal recommendations and query our knowledge graph to determine what restaurant the user needs to go to for said meals based on their dietary needs.
</div>